<a href="https://colab.research.google.com/github/MitziVite/machine_learning/blob/main/notebooks/starter_bikes2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
bikes.head()

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1/1/2011,0.0,3,13,3.0,3.0,0.7957,0.8,1,1,0,0
1,1/1/2011,1.0,8,30,1.7,1.7,0.8272,0.8,1,1,0,0
2,1/1/2011,2.0,5,26,1.9,1.9,0.8157,1.1,1,1,0,0
3,1/1/2011,3.0,3,9,2.5,2.5,0.7831,0.8,1,1,0,0
4,1/1/2011,4.0,0,1,2.0,2.0,0.8075,1.1,1,1,0,0


In [2]:
bikes["total_rentals"] = bikes["casual"] + bikes["registered"]

bikes["dteday"] = pd.to_datetime(bikes["dteday"])
bikes["day"] = bikes["dteday"].dt.day
bikes["month"] = bikes["dteday"].dt.month
bikes["year"] = bikes["dteday"].dt.year

features = [
    "hr", "temp_c", "feels_like_c", "hum", "windspeed",
    "weathersit", "season", "holiday", "workingday",
    "day", "month", "year"
]

X = bikes[features]
y = bikes["total_rentals"]


In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import regularizers

model = Sequential([
    Input(shape=(X_train.shape[1],)), # Use Input layer for specifying input shape
    Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.0005)), # Adjusted L2 strength
    Dropout(0.3), # Added Dropout layer
    Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.0005)), # Adjusted L2 strength
    Dropout(0.3), # Added Dropout layer
    Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.0005)), # Adjusted L2 strength
    Dense(1)
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss="mse", metrics=["mae"])

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=250,
    batch_size=64,
    verbose=1
)

# Evaluation
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f}")

# Predictions
bikes["predicted_total"] = model.predict(scaler.transform(X))

# Save only the first 384 predictions as 'total_rentals'
bikes["predicted_total"].head(384).to_csv("bike_predictions.csv", index=False, header=['total_rentals'])

Epoch 1/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 110124.8047 - mae: 227.9531 - val_loss: 70658.0547 - val_mae: 193.2129
Epoch 2/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 72655.0781 - mae: 194.0932 - val_loss: 68809.3516 - val_mae: 188.6392
Epoch 3/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 69637.1562 - mae: 187.4944 - val_loss: 56830.0117 - val_mae: 160.7608
Epoch 4/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 59911.3867 - mae: 169.6272 - val_loss: 48358.9258 - val_mae: 146.2763
Epoch 5/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 53917.1562 - mae: 154.6009 - val_loss: 45600.4883 - val_mae: 133.9125
Epoch 6/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 49602.5859 - mae: 146.5345 - val_loss: 42410.7695 - val_mae: 135.0581
Epoch 7/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 48698.7148 - mae: 145.2081 - val_loss: 41163.1172 - val_mae: 129.4498
Epoch 8/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 47371

# Task
Improve the model's performance in the notebook "/content/module04_biking_grading.ipynb" by iterating on the model architecture, hyperparameters, feature engineering, and regularization techniques, evaluating the results using the provided metrics (Within 5%, Within 10%, Within 20%, R^2, RMSE, MAE, MedAE) after each change, and summarizing the changes and results.

## Revisar la arquitectura del modelo

### Subtask:
Experimentar con un número diferente de capas densas y neuronas en cada capa.


**Reasoning**:
Modify the model architecture with a different number of dense layers and neurons, keeping the input shape and the output layer the same, and then train and evaluate the model.



In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss="mse", metrics=["mae"])

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=250,
    batch_size=64,
    verbose=1
)

# Evaluation
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f}")

# Predictions
bikes["predicted_total"] = model.predict(scaler.transform(X))

# Save only the first 384 predictions as 'total_rentals'
bikes["predicted_total"].head(384).to_csv("bike-predictions.csv", index=False, header=['total_rentals'])
print("✅ CSV export complete!")

Epoch 1/250


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1125/1125 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 110175.4062 - mae: 229.2974 - val_loss: 62159.2656 - val_mae: 172.8886
Epoch 2/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 53219.5312 - mae: 156.8279 - val_loss: 40562.3008 - val_mae: 129.8511
Epoch 3/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 38741.4180 - mae: 128.1618 - val_loss: 29645.1699 - val_mae: 112.0340
Epoch 4/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 27637.4531 - mae: 107.9575 - val_loss: 21807.0801 - val_mae: 95.0055
Epoch 5/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 20803.1855 - mae: 94.4149 - val_loss: 19348.3184 - val_mae: 92.7008
Epoch 6/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 17799.8555 - mae: 87.1144 - val_loss: 16980.7637 - val_mae: 84.7362
Epoch 7/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 16104.8828 - mae: 83.0080 - val_loss: 15006.5811 - val_mae: 80.0979
Epoch 8/250
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 14697.6016 - mae: 78.832